In [1]:
import spacy
from spacy import displacy
import pandas as pd
import numpy as np

In [2]:
# lisab NERi tõhjale 'de' mudelile, kuhu on juba cmd-ga vektorid lisatud

# nlp = spacy.load('../data/models/main/')
# nlp.add_pipe('ner')
# nlp.to_disk('../data/models/main/')

In [70]:
nlp = spacy.load('../data/models/main_2/model-best/')

In [7]:
nlp.pipeline

[('ner', <spacy.pipeline.ner.EntityRecognizer at 0x1fd8fc58fa0>)]

In [59]:
de_core_news = spacy.load('de_core_news_md')

In [9]:
df = pd.read_parquet('../data/processed/RZ_sample.parquet')

In [65]:
displacy_color_code = {'WEA': '#4cafd9',
                  'PER': '#ffb366',
                  'DAT': '#bf80ff',
                  'LOC': '#a88676'}

displacy_options = {'ents': ['WEA', 'PER', 'DAT', 'LOC'], 'colors': displacy_color_code}

def example():
    
    wea = 0
    
    while wea == 0:
        i = np.random.randint(len(df))
        text = df.full_text.iloc[i]
        year = df.year.iloc[i]
        doc = nlp(text)
        if 'WEA' in set([ent.label_ for ent in doc.ents]):
            wea = 1
        
    print(year, '\n')
    displacy.render(doc, style='ent', jupyter=True, options=displacy_options)

In [81]:
example()

1830 



In [3]:
import gensim

In [4]:
model = gensim.models.keyedvectors.KeyedVectors.load_word2vec_format('../data/models/word2vec_251021/w2v_model.txt')

In [5]:
model.most_similar('Wärme',topn=30)

[('Warme', 0.7943295240402222),
 ('wärmer', 0.6470202207565308),
 ('Temperatnr', 0.6255499124526978),
 ('ausstrahlende', 0.6227114200592041),
 ('Wirme', 0.6134321093559265),
 ('Gefrierpunkt', 0.6067052483558655),
 ('Wärm', 0.6052025556564331),
 ('Kälte', 0.6050214767456055),
 ('kälter', 0.5980657935142517),
 ('Sonnenwärme', 0.5904731750488281),
 ('Reaumur', 0.5872997045516968),
 ('wärmere', 0.5872786045074463),
 ('Kühle', 0.5867332816123962),
 ('Ausstrahlung', 0.5806692838668823),
 ('Temperatur', 0.5784103870391846),
 ('intensiv', 0.575846254825592),
 ('erwärmte', 0.564342200756073),
 ('erwärmt', 0.561190664768219),
 ('wärme', 0.561089277267456),
 ('intensiver', 0.559658944606781),
 ('Fahrenheit', 0.5547251105308533),
 ('kälteren', 0.5529605746269226),
 ('Külte', 0.5519946813583374),
 ('feuchter', 0.5495530962944031),
 ('Thermometers', 0.5482785105705261),
 ('erwärmende', 0.5480726361274719),
 ('kühlerer', 0.5478411912918091),
 ('Rêaumur', 0.5472375750541687),
 ('eisiger', 0.5452985763

In [82]:
from spacy.pipeline.spancat import DEFAULT_SPANCAT_MODEL

In [84]:
spancat = DEFAULT_SPANCAT_MODEL

In [86]:
spancat.

dict_items([('@architectures', 'spacy.SpanCategorizer.v1'), ('scorer', {'@layers': 'spacy.LinearLogistic.v1'}), ('reducer', {'@layers': 'spacy.mean_max_reducer.v1', 'hidden_size': 128}), ('tok2vec', {'@architectures': 'spacy.Tok2Vec.v1', 'embed': {'@architectures': 'spacy.MultiHashEmbed.v1', 'width': 96, 'rows': [5000, 2000, 1000, 1000], 'attrs': ['ORTH', 'PREFIX', 'SUFFIX', 'SHAPE'], 'include_static_vectors': False}, 'encode': {'@architectures': 'spacy.MaxoutWindowEncoder.v1', 'width': 96, 'window_size': 1, 'maxout_pieces': 3, 'depth': 4}})])